In [9]:
import numpy as np
import pandas as pd
from arch import arch_model
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
ef = pd.read_csv('sample_input.csv')
def evaluate():
    df = pd.read_csv('STOCK_INDEX.csv')
    
    actual_close = np.loadtxt('sample_close.txt')
    
    pred_close = predict_func(df)
    
    # Calculation of squared_error
    actual_close = np.array(actual_close)
    pred_close = np.array(pred_close)
    mean_square_error = np.mean(np.square(actual_close-pred_close))


    pred_prev = [df['Close'].iloc[-1]]
    pred_prev.append(pred_close[0])
    pred_curr = pred_close
    
    actual_prev = [df['Close'].iloc[-1]]
    actual_prev.append(actual_close[0])
    actual_curr = actual_close

    # Calculation of directional_accuracy
    pred_dir = np.array(pred_curr)[:, 0] - np.array(pred_prev[1])
    actual_dir = np.array(actual_curr) - np.array(actual_prev)
    dir_accuracy = np.mean((pred_dir*actual_dir)>0)*100

    print(f'Mean Square Error: {mean_square_error:.6f}\nDirectional Accuracy: {dir_accuracy:.1f}')
    
def predict_func(df):
    df.ffill(inplace=True)
    # Prepare the DataFrame with log returns
    df['log_returns'] = np.log(df['Close']).diff().dropna()
    # Create a scaler to normalize the data
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(df['Close'].values.reshape(-1, 1))
    # Split the data into training and test sets
    train_size = int(len(scaled_data))
    train_data = scaled_data[:train_size]
#     test_data = scaled_data[train_size:]

    # Prepare the training data for LSTM
    X_train_lstm, y_train_lstm = [], []
    lookback_lstm = 10  # Adjust the lookback window for LSTM

    for i in range(lookback_lstm, len(train_data)):
        X_train_lstm.append(train_data[i-lookback_lstm:i, 0])
        y_train_lstm.append(train_data[i, 0])

    X_train_lstm, y_train_lstm = np.array(X_train_lstm), np.array(y_train_lstm)

    # Reshape the input data for LSTM
    X_train_lstm = np.reshape(X_train_lstm, (X_train_lstm.shape[0], X_train_lstm.shape[1], 1))

    # Create and fit the LSTM model
    lstm_model = Sequential()
    lstm_model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train_lstm.shape[1], 1)))
    lstm_model.add(LSTM(units=50))
    lstm_model.add(Dense(units=1))
    lstm_model.compile(optimizer='adam', loss='mean_squared_error')
    lstm_model.fit(X_train_lstm, y_train_lstm, epochs=10, batch_size=32)

    ef.ffill(inplace=True)
    # Prepare the DataFrame with log returns
    ef['log_returns'] = np.log(ef['Close']).diff().dropna()
    # Create a scaler to normalize the data
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(ef['Close'].values.reshape(-1, 1))
    # Split the data into training and test sets
    test_size = int(len(scaled_data))
#     train_data = scaled_data[:train_size]
    test_data = scaled_data[:test_size]
    
    # Prepare the test data for LSTM
    inputs_lstm = ef['Close'].values[len(ef) - len(test_data) - lookback_lstm:]
    inputs_lstm = inputs_lstm.reshape(-1, 1)
    inputs_lstm = scaler.transform(inputs_lstm)

    X_test_lstm = []

    for i in range(lookback_lstm, len(inputs_lstm)):
        X_test_lstm.append(inputs_lstm[i-lookback_lstm:i, 0])
    
    X_test_lstm = np.array(X_test_lstm)
    X_test_lstm = np.reshape(X_test_lstm, (X_test_lstm.shape[0], X_test_lstm.shape[1], 1))


    # Make predictions using LSTM model
    predicted_prices_lstm = lstm_model.predict(X_test_lstm)
    predicted_prices_lstm = scaler.inverse_transform(predicted_prices_lstm)

    # Prepare the test data for EMA
    inputs_ema = ef['Close'].values[len(ef) - len(test_data) - lookback_lstm:]
    inputs_ema = inputs_ema.reshape(-1, 1)
    inputs_ema = scaler.transform(inputs_ema)

    # Calculate EMA using alpha 0.75
    ema = [inputs_ema[0]]  # Initialize the first EMA value as the first data point
    alpha = 0.85
    for i in range(1, len(inputs_ema)):
        ema_value = alpha * inputs_ema[i] + (1 - alpha) * ema[i - 1]
        ema.append(ema_value)

    ema = np.array(ema)
    ema = scaler.inverse_transform(ema)

    # Adjust the shapes of predicted_prices_lstm and ema arrays
    predicted_prices_lstm = predicted_prices_lstm[-len(ema):]

    # Combine the predictions
    combined_predictions = 0.05 * predicted_prices_lstm[-10:] + 0.95 * ema[-10:]
    # Return the combined predictions for the next two days
    next_two_days = combined_predictions[-2:]
    return next_two_days

if(__name__ == '__main__'):
    evaluate()

Epoch 1/10
84/84 [==============================] - 8s 18ms/step - loss: 0.0185
Epoch 2/10
84/84 [==============================] - 1s 17ms/step - loss: 4.0266e-04
Epoch 3/10
84/84 [==============================] - 1s 17ms/step - loss: 4.0308e-04
Epoch 4/10
84/84 [==============================] - 1s 18ms/step - loss: 3.8157e-04
Epoch 5/10
84/84 [==============================] - 2s 18ms/step - loss: 3.7853e-04
Epoch 6/10
84/84 [==============================] - 2s 18ms/step - loss: 3.9504e-04
Epoch 7/10
84/84 [==============================] - 2s 18ms/step - loss: 3.9471e-04
Epoch 8/10
84/84 [==============================] - 1s 18ms/step - loss: 3.6105e-04
Epoch 9/10
84/84 [==============================] - 1s 18ms/step - loss: 3.5409e-04
Epoch 10/10
84/84 [==============================] - 1s 18ms/step - loss: 3.5389e-04


IndexError: tuple index out of range

In [11]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

df = pd.read_csv('STOCK_INDEX.csv')
ef = pd.read_csv('sample_input.csv')

def evaluate():
    actual_close = np.loadtxt('sample_close.txt')
    pred_close = predict_func(df, ef)

    # Calculation of squared_error
    mean_square_error = np.mean(np.square(actual_close - pred_close))

    pred_prev = [df['Close'].iloc[-1]]
    pred_prev.append(pred_close[0])
    pred_curr = pred_close

    actual_prev = [df['Close'].iloc[-1]]
    actual_prev.append(actual_close[0])
    actual_curr = actual_close

    # Calculation of directional_accuracy
    pred_dir = np.array(pred_curr)[:, 0] - np.array(pred_prev[1])
    actual_dir = np.array(actual_curr) - np.array(actual_prev)
    dir_accuracy = np.mean((pred_dir * actual_dir) > 0) * 100

    print(f'Mean Square Error: {mean_square_error:.6f}\nDirectional Accuracy: {dir_accuracy:.1f}')

def predict_func(train_df, test_df):
    train_df.ffill(inplace=True)
    # Prepare the DataFrame with log returns
    train_df['log_returns'] = np.log(train_df['Close']).diff().dropna()
    # Create a scaler to normalize the data
    scaler = MinMaxScaler()
    scaled_train_data = scaler.fit_transform(train_df['Close'].values.reshape(-1, 1))

    # Prepare the training data for LSTM
    X_train_lstm, y_train_lstm = [], []
    lookback_lstm = 10  # Adjust the lookback window for LSTM

    for i in range(lookback_lstm, len(scaled_train_data)):
        X_train_lstm.append(scaled_train_data[i - lookback_lstm:i, 0])
        y_train_lstm.append(scaled_train_data[i, 0])

    X_train_lstm, y_train_lstm = np.array(X_train_lstm), np.array(y_train_lstm)

    # Reshape the input data for LSTM
    X_train_lstm = np.reshape(X_train_lstm, (X_train_lstm.shape[0], X_train_lstm.shape[1], 1))

    # Create and fit the LSTM model
    lstm_model = Sequential()
    lstm_model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train_lstm.shape[1], 1)))
    lstm_model.add(LSTM(units=50))
    lstm_model.add(Dense(units=1))
    lstm_model.compile(optimizer='adam', loss='mean_squared_error')
    lstm_model.fit(X_train_lstm, y_train_lstm, epochs=10, batch_size=32)

    test_df.ffill(inplace=True)
    # Prepare the DataFrame with log returns
    test_df['log_returns'] = np.log(test_df['Close']).diff().dropna()
    # Normalize the test data using the scaler from training
    scaled_test_data = scaler.transform(test_df['Close'].values.reshape(-1, 1))

    # Prepare the test data for LSTM
    inputs_lstm = scaled_test_data[len(test_df) - lookback_lstm:]
    inputs_lstm = inputs_lstm.reshape(-1, 1)
    inputs_lstm = scaler.transform(inputs_lstm)

    X_test_lstm = []

    for i in range(lookback_lstm, len(inputs_lstm)):
        X_test_lstm.append(inputs_lstm[i - lookback_lstm:i, 0])

    X_test_lstm = np.array(X_test_lstm)
    X_test_lstm = np.reshape(X_test_lstm, (X_test_lstm.shape[0], lookback_lstm, 1))


    # Make predictions using LSTM model
    predicted_prices_lstm = lstm_model.predict(X_test_lstm)
    predicted_prices_lstm = scaler.inverse_transform(predicted_prices_lstm)

    # Prepare the test data for EMA
    inputs_ema = scaled_test_data[len(test_df) - lookback_lstm:]
    inputs_ema = inputs_ema.reshape(-1, 1)

    # Calculate EMA using alpha 0.75
    ema = [inputs_ema[0]]  # Initialize the first EMA value as the first data point
    alpha = 0.85
    for i in range(1, len(inputs_ema)):
        ema_value = alpha * inputs_ema[i] + (1 - alpha) * ema[i - 1]
        ema.append(ema_value)

    ema = np.array(ema)
    ema = scaler.inverse_transform(ema)

    # Adjust the shapes of predicted_prices_lstm and ema arrays
    predicted_prices_lstm = predicted_prices_lstm[-len(ema):]

    # Combine the predictions
    combined_predictions = 0.05 * predicted_prices_lstm[-10:, 0] + 0.95 * ema[-10:, 0]
    # Return the combined predictions for the next two days
    next_two_days = combined_predictions[-2:]
    return next_two_days

if __name__ == '__main__':
    evaluate()


Epoch 1/10
84/84 [==============================] - 8s 22ms/step - loss: 0.0110
Epoch 2/10
84/84 [==============================] - 2s 21ms/step - loss: 3.9604e-04
Epoch 3/10
84/84 [==============================] - 2s 21ms/step - loss: 4.1955e-04
Epoch 4/10
84/84 [==============================] - 2s 21ms/step - loss: 4.0243e-04
Epoch 5/10
84/84 [==============================] - 2s 21ms/step - loss: 4.0805e-04
Epoch 6/10
84/84 [==============================] - 2s 21ms/step - loss: 4.0548e-04
Epoch 7/10
84/84 [==============================] - 2s 21ms/step - loss: 3.8226e-04
Epoch 8/10
84/84 [==============================] - 2s 21ms/step - loss: 3.6397e-04
Epoch 9/10
84/84 [==============================] - 2s 21ms/step - loss: 3.8894e-04
Epoch 10/10
84/84 [==============================] - 2s 21ms/step - loss: 3.7934e-04


ValueError: Unexpected result of `predict_function` (Empty batch_outputs). Please use `Model.compile(..., run_eagerly=True)`, or `tf.config.run_functions_eagerly(True)` for more information of where went wrong, or file a issue/bug to `tf.keras`.

Epoch 1/10
84/84 [==============================] - 8s 20ms/step - loss: 0.0087
Epoch 2/10
84/84 [==============================] - 2s 20ms/step - loss: 3.8722e-04
Epoch 3/10
84/84 [==============================] - 2s 20ms/step - loss: 3.8972e-04
Epoch 4/10
84/84 [==============================] - 2s 20ms/step - loss: 3.6669e-04
Epoch 5/10
84/84 [==============================] - 2s 21ms/step - loss: 3.6663e-04
Epoch 6/10
84/84 [==============================] - 2s 21ms/step - loss: 3.6493e-04
Epoch 7/10
84/84 [==============================] - 2s 21ms/step - loss: 3.6543e-04
Epoch 8/10
84/84 [==============================] - 2s 20ms/step - loss: 3.5002e-04
Epoch 9/10
84/84 [==============================] - 2s 21ms/step - loss: 3.5857e-04
Epoch 10/10
1/1 [==============================] - 1s 1s/step


IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [30]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

df = pd.read_csv('STOCK_INDEX.csv')
ef = pd.read_csv('sample_input.csv')

def evaluate():
    actual_close = np.loadtxt('sample_close.txt')
    pred_close = predict_func(df, ef)

    # Calculation of squared_error
#     mean_square_error = np.mean(np.square(actual_close - pred_close))

#     pred_prev = [df['Close'].iloc[-1]]
#     pred_prev.append(pred_close[0])
#     pred_curr = pred_close

#     actual_prev = [df['Close'].iloc[-1]]
#     actual_prev.append(actual_close[0])
#     actual_curr = actual_close

#     # Calculation of directional_accuracy
#     pred_dir = np.array(pred_curr) - np.array(pred_prev[1])
#     actual_dir = np.array(actual_curr) - np.array(actual_prev)
#     dir_accuracy = np.mean((pred_dir * actual_dir) > 0) * 100

#     print(f'Mean Square Error: {mean_square_error:.6f}\nDirectional Accuracy: {dir_accuracy:.1f}')

def predict_func(train_df, test_df):
    
    train_df.ffill(inplace=True)
    # Prepare the DataFrame with log returns
    train_df['log_returns'] = np.log(train_df['Close']).diff().dropna()
    # Create a scaler to normalize the data
    scaler = MinMaxScaler()
    scaled_train_data = scaler.fit_transform(train_df['Close'].values.reshape(-1, 1))

    # Prepare the training data for LSTM
    X_train_lstm, y_train_lstm = [], []
    lookback_lstm = 10  # Adjust the lookback window for LSTM

    for i in range(lookback_lstm, len(scaled_train_data)):
        X_train_lstm.append(scaled_train_data[i - lookback_lstm:i, 0])
        y_train_lstm.append(scaled_train_data[i, 0])

    X_train_lstm, y_train_lstm = np.array(X_train_lstm), np.array(y_train_lstm)

    # Reshape the input data for LSTM
    X_train_lstm = np.reshape(X_train_lstm, (X_train_lstm.shape[0], X_train_lstm.shape[1], 1))

    # Create and fit the LSTM model
    lstm_model = Sequential()
    lstm_model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train_lstm.shape[1], 1)))
    lstm_model.add(LSTM(units=50))
    lstm_model.add(Dense(units=1))
    lstm_model.compile(optimizer='adam', loss='mean_squared_error')
    lstm_model.fit(X_train_lstm, y_train_lstm, epochs=100, batch_size=64)
    
    lstm_model.save('model_final.m1')

if __name__ == '__main__':
    evaluate()


Epoch 1/100
42/42 [==============================] - 9s 43ms/step - loss: 0.0185
Epoch 2/100
42/42 [==============================] - 2s 43ms/step - loss: 5.3149e-04
Epoch 3/100
42/42 [==============================] - 2s 42ms/step - loss: 3.7185e-04
Epoch 4/100
42/42 [==============================] - 2s 42ms/step - loss: 3.6647e-04
Epoch 5/100
42/42 [==============================] - 2s 41ms/step - loss: 3.6393e-04
Epoch 6/100
42/42 [==============================] - 2s 41ms/step - loss: 3.6480e-04
Epoch 7/100
42/42 [==============================] - 2s 41ms/step - loss: 3.6123e-04
Epoch 8/100
42/42 [==============================] - 2s 41ms/step - loss: 3.6566e-04
Epoch 9/100
42/42 [==============================] - 2s 42ms/step - loss: 3.6192e-04
Epoch 10/100
42/42 [==============================] - 2s 41ms/step - loss: 3.7323e-04
Epoch 11/100
42/42 [==============================] - 2s 42ms/step - loss: 3.4945e-04
Epoch 12/100
42/42 [==============================] - 2s 41ms/step 

42/42 [==============================] - 2s 42ms/step - loss: 1.1548e-04
Epoch 97/100
42/42 [==============================] - 2s 45ms/step - loss: 1.1966e-04
Epoch 98/100
42/42 [==============================] - 2s 45ms/step - loss: 1.2439e-04
Epoch 99/100
42/42 [==============================] - 2s 47ms/step - loss: 1.2920e-04
Epoch 100/100
42/42 [==============================] - 2s 50ms/step - loss: 1.1501e-04
INFO:tensorflow:Assets written to: model_final.m1\assets


INFO:tensorflow:Assets written to: model_final.m1\assets


In [ ]:
test_df.ffill(inplace=True)
    # Prepare the DataFrame with log returns
    test_df['log_returns'] = np.log(test_df['Close']).diff().dropna()
    # Normalize the test data using the scaler from training
    scaled_test_data = scaler.transform(test_df['Close'].values.reshape(-1, 1))

    # Prepare the test data for LSTM
    inputs_lstm = scaled_test_data[-lookback_lstm:]
    inputs_lstm = inputs_lstm.reshape(1, -1, 1)

    # Make predictions using LSTM model
    predicted_prices_lstm = lstm_model.predict(inputs_lstm)
    predicted_prices_lstm = scaler.inverse_transform(predicted_prices_lstm)

    # Prepare the test data for EMA
    inputs_ema = scaled_test_data[-lookback_lstm:]
    inputs_ema = inputs_ema.reshape(-1, 1)

    # Calculate EMA using alpha 0.75
    ema = [inputs_ema[0]]  # Initialize the first EMA value as the first data point
    alpha = 0.85
    for i in range(1, len(inputs_ema)):
        ema_value = alpha * inputs_ema[i] + (1 - alpha) * ema[i - 1]
        ema.append(ema_value)

    ema = np.array(ema)
    ema = scaler.inverse_transform(ema)

    # Adjust the shapes of predicted_prices_lstm and ema arrays
    predicted_prices_lstm = predicted_prices_lstm[-len(ema):]

    # Combine the predictions
    combined_predictions = 0.05 * predicted_prices_lstm[-10:, 0] + 0.95 * ema[-10:, 0]
    # Return the combined predictions for the next two days
    next_two_days = combined_predictions[-2:]
    return next_two_days

In [29]:
    pred_prev = [df['Close'].iloc[-1]]
    pred_prev
    pred_prev.size()


AttributeError: 'list' object has no attribute 'size'